<a href="https://colab.research.google.com/github/SushmitaSingh96/Sound_Recognition_-_Cancellation/blob/main/FutureWork_Training_deepfilternet_on_own_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#pip install -r requirements.txt

In [2]:
#example dataset contains baby sounds
!git clone https://github.com/gveres/donateacry-corpus.git

fatal: destination path 'donateacry-corpus' already exists and is not an empty directory.


In [3]:
#to prepare the data move all WAV files from source folders to the "baby_sounds" folder

!mv /content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/belly_pain/*.wav /content/babySounds/
!mv /content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/burping/*.wav /content/babySounds/
!mv /content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/discomfort/*.wav /content/babySounds/
!mv /content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/hungry/*.wav /content/babySounds/
!mv /content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/tired/*.wav /content/babySounds/


mv: cannot stat '/content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/belly_pain/*.wav': No such file or directory
mv: cannot stat '/content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/burping/*.wav': No such file or directory
mv: cannot stat '/content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/discomfort/*.wav': No such file or directory
mv: cannot stat '/content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/hungry/*.wav': No such file or directory
mv: cannot stat '/content/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/tired/*.wav': No such file or directory


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

import csv

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile
import h5py
import librosa
import soundfile as sf
import pydub
from pydub import AudioSegment


In [5]:
#conversion of all the baby sounds to 48khz sampling rate
from pydub import AudioSegment
import os

input_folder = "/content/babySounds"
output_folder = "/content/babySounds48k"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through all WAV files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".wav"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Load the audio file
        audio = AudioSegment.from_file(input_path, format="wav")

        # Set the sample rate to 48kHz
        audio = audio.set_frame_rate(48000)

        # Convert to mono if not already
        if audio.channels > 1:
            audio = audio.set_channels(1)

        # Export as WAV file to the output folder
        audio.export(output_path, format="wav")


**Creation of train, validate, test sets of the sounds in babySounds48k folder**

In [6]:
import os
import random
import shutil

# Set the paths for the original dataset folder and the destination folders
dataset_folder = "/content/babySounds48k"
training_folder = "/content/babysoundstrain/training_set"
validation_folder = "/content/babysoundstrain/validation_set"
testing_folder = "/content/babysoundstrain/testing_set"

# Set the split ratios for training, validation, and testing
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15

# Create the destination folders if they don't exist
os.makedirs(training_folder, exist_ok=True)
os.makedirs(validation_folder, exist_ok=True)
os.makedirs(testing_folder, exist_ok=True)

# Get the list of files in the dataset folder
files = os.listdir(dataset_folder)

# Shuffle the files randomly
random.shuffle(files)

# Calculate the number of files for each split
num_files = len(files)
num_train = int(num_files * train_ratio)
num_valid = int(num_files * valid_ratio)

# Move the files to the respective folders based on the split ratios
for i, file in enumerate(files):
    if i < num_train:
        shutil.move(os.path.join(dataset_folder, file), os.path.join(training_folder, file))
    elif i < num_train + num_valid:
        shutil.move(os.path.join(dataset_folder, file), os.path.join(validation_folder, file))
    else:
        shutil.move(os.path.join(dataset_folder, file), os.path.join(testing_folder, file))

#Creation of text files and HDF5 files for train, test and validation sets

In [7]:
!mkdir dataset_txts

mkdir: cannot create directory ‘dataset_txts’: File exists


In [8]:
#creation of txt file for training
import os

directory_path = "/content/babysoundstrain/training_set"  # Replace with the path to your folder
output_file_path = "/content/dataset_txts/training_set.txt"  # Specify the desired output file path

def write_file_paths_to_txt(directory_path, output_file_path):
    # Get a list of all files in the specified directory
    file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]

    # Write the file paths to a text file
    with open(output_file_path, 'w') as output_file:
        for file_path in file_paths:
            output_file.write(file_path + '\n')
write_file_paths_to_txt(directory_path, output_file_path)

In [9]:
#creation of txt file for testing
import os

directory_path = "/content/babysoundstrain/testing_set"  # Replace with the path to your folder
output_file_path = "/content/dataset_txts/testing_set.txt"  # Specify the desired output file path

def write_file_paths_to_txt(directory_path, output_file_path):
    # Get a list of all files in the specified directory
    file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]

    # Write the file paths to a text file
    with open(output_file_path, 'w') as output_file:
        for file_path in file_paths:
            output_file.write(file_path + '\n')
write_file_paths_to_txt(directory_path, output_file_path)

In [10]:
#creation of txt file for validation
import os

directory_path = "/content/babysoundstrain/validation_set"  # Replace with the path to your folder
output_file_path = "/content/dataset_txts/validation_set.txt"  # Specify the desired output file path

def write_file_paths_to_txt(directory_path, output_file_path):
    # Get a list of all files in the specified directory
    file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]

    # Write the file paths to a text file
    with open(output_file_path, 'w') as output_file:
        for file_path in file_paths:
            output_file.write(file_path + '\n')
write_file_paths_to_txt(directory_path, output_file_path)

In [11]:
mkdir hdf5_data

mkdir: cannot create directory ‘hdf5_data’: File exists


we cd to location in colab where deepfilternet[train] is installed

In [12]:
cd /usr/local/lib/python3.10/dist-packages/

/usr/local/lib/python3.10/dist-packages


In [13]:
!python df/scripts/prepare_data.py --sr 48000 speech /content/dataset_txts/training_set.txt /content/hdf5_data/TRAIN_SET_SPEECH.hdf5

/usr/local/lib/python3.10/dist-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData
2024-04-09 14:02:53 | INFO     | DF | Running on torch 2.2.1+cu121
2024-04-09 14:02:53 | INFO     | DF | Running on host 257ee3dd51b9
fatal: not a git repository (or any of the parent directories): .git
2024-04-09 14:02:53 | INFO     | DF | Using speech working directory /content/dataset_txts
2024-04-09 14:02:53 | INFO     | DF | Checking all audio files complete
/usr/local/lib/python3.10/dist-packages/df/scripts/prepare_data.py:46: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5.File(file_name, "a", libver="latest", swmr=True) as f, torch.no_grad():
2024-04-09 14:02:53 | INFO     | DF | Found existing group speech
/usr/local/lib/python3.10/dist-packages/torch/utils/data

In [14]:
!python df/scripts/prepare_data.py --sr 48000 speech /content/dataset_txts/testing_set.txt /content/hdf5_data/TEST_SET_SPEECH.hdf5

/usr/local/lib/python3.10/dist-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData
2024-04-09 14:03:14 | INFO     | DF | Running on torch 2.2.1+cu121
2024-04-09 14:03:14 | INFO     | DF | Running on host 257ee3dd51b9
fatal: not a git repository (or any of the parent directories): .git
2024-04-09 14:03:14 | INFO     | DF | Using speech working directory /content/dataset_txts
2024-04-09 14:03:14 | INFO     | DF | Checking all audio files complete
/usr/local/lib/python3.10/dist-packages/df/scripts/prepare_data.py:46: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5.File(file_name, "a", libver="latest", swmr=True) as f, torch.no_grad():
2024-04-09 14:03:14 | INFO     | DF | Found existing group speech
/usr/local/lib/python3.10/dist-packages/torch/utils/data

In [15]:
!python df/scripts/prepare_data.py --sr 48000 speech /content/dataset_txts/validation_set.txt /content/hdf5_data/VALID_SET_SPEECH.hdf5

/usr/local/lib/python3.10/dist-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData
2024-04-09 14:03:24 | INFO     | DF | Running on torch 2.2.1+cu121
2024-04-09 14:03:24 | INFO     | DF | Running on host 257ee3dd51b9
fatal: not a git repository (or any of the parent directories): .git
2024-04-09 14:03:24 | INFO     | DF | Using speech working directory /content/dataset_txts
2024-04-09 14:03:24 | INFO     | DF | Checking all audio files complete
/usr/local/lib/python3.10/dist-packages/df/scripts/prepare_data.py:46: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5.File(file_name, "a", libver="latest", swmr=True) as f, torch.no_grad():
2024-04-09 14:03:24 | INFO     | DF | Found existing group speech
/usr/local/lib/python3.10/dist-packages/torch/utils/data

In [16]:
cd /content

/content


# Preparation of configuration file for training
Due to a lack of proper dataset we could only prepare the configuration file for the SPEECH data. To complete the training, we need NOISE as well as RIR datasets.

In [17]:
%%writefile dataset.cfg
{
  "train": [
    [
      "TRAIN_SET_SPEECH.hdf5",
      1.0
    ]
  ],
  "valid": [
    [
      "VALID_SET_SPEECH.hdf5",
      1.0
    ]
  ],
  "test": [
    [
      "TEST_SET_SPEECH.hdf5",
      1.0
    ]
  ]
}

Overwriting dataset.cfg
